<a href="https://colab.research.google.com/github/KurniaKhaikal/Cifar10k_TransferLearning/blob/main/TransferLearning_Cifar10_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import copy
import numpy as np
import os
import json
import cv2
import sklearn.metrics as metric
from tensorflow.keras.models import Model
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"khaikal","key":"9e492a7553b8e713debda6f4f6810151"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d oxcdcd/cifar10

 96% 169M/175M [00:01<00:00, 138MB/s]
100% 175M/175M [00:01<00:00, 134MB/s]


In [ ]:
import os
import zipfile

local_zip = '/content/cifar10.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/')
zip_ref.close()

In [ ]:
!pip install split-folders

In [ ]:
import splitfolders


splitfolders.ratio("/content/cifar10/test", output="/content/drive/MyDrive/Proposal2/Cifar10/Cifar_dataset/cifar10/Base2",
    seed=1337, ratio=(.9, .1), group_prefix=None, move=False)

Copying files: 10000 files [01:49, 91.09 files/s]


In [5]:
def euclidean(a, b):
	return np.linalg.norm(a - b)

In [6]:
def perform_search(query_features, indexed_train, max_results=5):
	retrieved = []
	for idx in range(0, len(indexed_train["features"])):
		distance = euclidean(query_features, indexed_train["features"][idx])
		retrieved.append((distance, idx))
	retrieved = sorted(retrieved)[:max_results]
	return retrieved

In [10]:
base_dataset = "/content/drive/MyDrive/Proposal2/Cifar10/Cifar_dataset/cifar10/Base2"
class_dir = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
IMAGE_SIZE = (224, 224)

In [11]:
# indexing file images
dataset_test = []
for class_item in class_dir:
    cur_dir = os.path.join(base_dataset, 'val', class_item)
    for file in os.listdir(cur_dir):
        dataset_test.append(os.path.join(cur_dir, file))

In [12]:
print("len to retrieving:", len(dataset_test))

len to retrieving: 1000


In [13]:
dataset_test

['/content/drive/MyDrive/Proposal2/Cifar10/Cifar_dataset/cifar10/Base2/val/airplane/428_airplane.png',
 '/content/drive/MyDrive/Proposal2/Cifar10/Cifar_dataset/cifar10/Base2/val/airplane/667_airplane.png',
 '/content/drive/MyDrive/Proposal2/Cifar10/Cifar_dataset/cifar10/Base2/val/airplane/7774_airplane.png',
 '/content/drive/MyDrive/Proposal2/Cifar10/Cifar_dataset/cifar10/Base2/val/airplane/5043_airplane.png',
 '/content/drive/MyDrive/Proposal2/Cifar10/Cifar_dataset/cifar10/Base2/val/airplane/4748_airplane.png',
 '/content/drive/MyDrive/Proposal2/Cifar10/Cifar_dataset/cifar10/Base2/val/airplane/8476_airplane.png',
 '/content/drive/MyDrive/Proposal2/Cifar10/Cifar_dataset/cifar10/Base2/val/airplane/422_airplane.png',
 '/content/drive/MyDrive/Proposal2/Cifar10/Cifar_dataset/cifar10/Base2/val/airplane/3971_airplane.png',
 '/content/drive/MyDrive/Proposal2/Cifar10/Cifar_dataset/cifar10/Base2/val/airplane/7103_airplane.png',
 '/content/drive/MyDrive/Proposal2/Cifar10/Cifar_dataset/cifar10/Ba

In [14]:
print("[INFO] load images Corel-Image 1k dataset...")
#  load images
val_images = []
for image_path in dataset_test:
    if ".png" or ".jpeg" in image_path:
        image = cv2.imread(image_path)
        image = cv2.resize(image, IMAGE_SIZE)
        val_images.append(image)

[INFO] load images Corel-Image 1k dataset...


In [15]:
print("[INFO] normalization...")
val_x = np.array(val_images).astype("float32") / 255.0

[INFO] normalization...


In [17]:
from tensorflow.keras.applications import ResNet50
modelRestNet = ResNet50(weights="imagenet")
model = Model(inputs=modelRestNet.input, outputs=modelRestNet.output)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [18]:
features = model.predict(val_x)

In [19]:
with open('/content/drive/MyDrive/Feature Extraction/ResNet_feature_extraction.json') as f:
  training_indexed = json.load(f)

In [20]:
query_indexes = list(range(0, val_x.shape[0]))
label_builder = list(np.unique(training_indexed["labels"]))
class_builder = {label_unique:[] for label_unique in label_builder}
recalls = copy.deepcopy(class_builder)
precisions = copy.deepcopy(class_builder)
# loop over the testing indexes
for i in query_indexes:
    queryFeatures = features[i]
    results = perform_search(queryFeatures, training_indexed, max_results=5)
    labels_ret = [training_indexed["labels"][r[1]] for r in results]
    label_true = dataset_test[i].split("/")[10]
    label_trues = [label_true for _ in labels_ret]
    recall = metric.recall_score(label_trues, labels_ret, average='weighted')
    precision = metric.precision_score(label_trues, labels_ret, average='weighted')
    recalls[label_true].append(recall)
    precisions[label_true].append(precision)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and 

In [21]:
print("recall values:")
comb_recall, comb_precision = [], []
for key in recalls.keys():
    average_val = np.average(recalls[key])
    print(key, average_val)
    comb_recall.append(average_val)
print("combined recall", np.average(comb_recall))

print("\nprecision values:")
for key in precisions.keys():
    average_val = np.average(precisions[key])
    print(key, average_val)
    comb_precision.append(average_val)
print("combined precision", np.average(comb_precision))

recall values:
airplane 0.24
automobile 0.20199999999999996
bird 0.162
cat 0.144
deer 0.26799999999999996
dog 0.16399999999999998
frog 0.24599999999999994
horse 0.21599999999999997
ship 0.20999999999999996
truck 0.21600000000000003
combined recall 0.2068

precision values:
airplane 0.63
automobile 0.58
bird 0.56
cat 0.53
deer 0.75
dog 0.53
frog 0.72
horse 0.66
ship 0.66
truck 0.64
combined precision 0.626
